# QFin Trading Platform Tutorial

### Installing

In [ ]:
pip install qfinuwa

In [1]:
from qfinuwa import Indicators
import pandas as pd

## Indicators

To make a custom indicator class, extend the base ``Indicator`` class. 

In your class you can implement your indicators that can be accessed at runtime. You can do this by defining a function that takes in data and custom keyword arguments (that act as hyperparameters) and returns a dictionary of indicator names to values. Each function effectively groups the indicators together.

During runtime the values of all your indicators will be contained in the ``indicators`` object, which is a dictionary of indicator names to ``np.array`` values.

```py
indicator = {'custom_indicatorA': ...,
              'custom_indicatorB': ...,
              ...
            }
```

There are two types of indicators you can implement:

### MultiIndicator

Creates an indicator on each stock.

- input: A single stock dataframe, ``stock``
- output: A dictionary of indicators to values

- during runtime: ``indicators['custom_indicator']`` is a dictionary of indicators on each stock. Access the indicator signal for ``stockA`` by ``indicators['custom_indicator']['stockA']``

## SingleIndicator

Creates a single indicator for all stocks.

- input: A dictionary of stock names to dataframes, ``stocks``
- output: A dictionary of indicators to values

- during runtime: ``indicators['custom_indicator']`` is a single ``np.array``.


In [43]:

# Example
class CustomIndicators(Indicators):
    
    @Indicators.MultiIndicator
    def bollinger_bands(self, stock, BOLLINGER_WIDTH = 2, WINDOW_SIZE = 100):

        mid_price = (stock['high'] + stock['low']) / 2
        rolling_mid = mid_price.rolling(WINDOW_SIZE).mean()
        rolling_std = mid_price.rolling(WINDOW_SIZE).std()

        return {"upper_bollinger": rolling_mid + BOLLINGER_WIDTH*rolling_std,
                "lower_bollinger": rolling_mid - BOLLINGER_WIDTH*rolling_std}
    
    # @Indicators.SingleIndicator
    # def etf(self, stocks, a = 0.2, b = 0.2, c = 0.05, d=0.05):

    #     return {'etf':  a*stocks['AAPL']['close'] + \
    #                     b*stocks['MSFT']['close'] + \
    #                     c*stocks['GOOGL']['close'] + \
    #                     d*stocks['NVDA']['close']}

### Manual Testing

Test your indicators manually.

In [3]:
ci = CustomIndicators()
df1 = pd.read_csv('data/VGT.csv')
df2 = pd.read_csv('data/IYW.csv')

# test bollinger indicator
multiindicator_output = ci.bollinger_bands(df1)

# test etf indicator
# singleindicator_output = ci.etf({'df1': df1, 'df2': df2}, a = 0.5, b = 0.1)

## Strategy Class

Similar to the ``Indicators`` class, we can implement a strategy by extending ``qfinuwa.Strategy``.

The null strategy would be implemented as follows:

In [4]:
from qfinuwa import Strategy

In [5]:

class MyStrategy(Strategy):
    
    def __init__(self):
        return
    
    def on_data(self, data, indicators, portfolio):
       return
    
    def on_finish(self):
        return

The strategy offers custom implementation of three functions.

- ``__init``: called on creation of the strategy
- ``on_data``: called every data tick 
- ``on_finish``: return an object to be appended to the results

### ``__init__``

You can create variables here that can be accessed by ``on_data`` at runtime. 

Similar to each of your indicator functions, ``kwargs`` can be specified as hyperparamters (more on this later).

### ``on_data``

``on_data`` expects 3 additional arugments that will be populated at runtime.

- ``data``: the historic data up to this point
- ``indicators``: the historic value of your custom indicators
- ``portfolio``: an object that manages your position

### ``on_finish``

Should return an object that will be added to the results. An example use case is if you want to measure some statistic over time, and the backtester doesn't offer it, so you record it yourself during runtime and return it via this function.

### Example 1 - BollingerBand

In [6]:
class BollingerBands(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
        self.n_failed_orders = 0
    
    def on_data(self, prices, indicators, portfolio):

        # If current price is below lower Bollinger Band, enter a long position
        for stock in portfolio.stocks:

            if(prices['close'][stock][-1] < indicators['lower_bollinger'][stock][-1]):
                order_success = portfolio.order(stock, quantity=self.quantity)
                if not order_success:
                    self.n_failed_orders += 1
            
            # If current price is above upper Bollinger Band, enter a short position
            if(prices['close'][stock][-1] > indicators['upper_bollinger'][stock][-1]):
                order_success = portfolio.order(stock, quantity=-self.quantity)
                if not order_success:
                    self.n_failed_orders += 1

    def on_finish(self):
        # Added to results object - access using result.on_finish
        return self.n_failed_orders

## Example 2 - RandomStrategy

In [7]:
import random

class RandomStrategy(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
        self.n_failed_orders = 0
    
    def on_data(self, prices, indicators, portfolio):

        # this strategy doesn't use prices or indicators - see BollingerBand Example

        s = random.choice(portfolio.stocks)
        q = random.randint(-self.quantity, self.quantity)

        order_success = portfolio.order(s, quantity=q)
        if not order_success:
            self.n_failed_orders += 1

    def on_finish(self):
        # Added to results object - access using result.on_finish
        return self.n_failed_orders

In [52]:
class MyStrategy(Strategy):
    
    def __init__(self, quantity=5):
        self.quantity = quantity
        self.n_failed_orders = 0
    
    def on_data(self, prices, indicators, portfolio):

        # If current price is below lower Bollinger Band, enter a long position
        for stock in portfolio.stocks:

            if(prices['close'][stock][-1] < indicators['lower_bollinger'][stock][-1]):
                portfolio.order(stock, quantity=-self.quantity)
            
            # If current price is above upper Bollinger Band, enter a short position
            if(prices['close'][stock][-1] > indicators['upper_bollinger'][stock][-1]):
                portfolio.order(stock, quantity=self.quantity)

    def on_finish(self):
        # Added to results object - access using result.on_finish
        return

## Portfolio Class

To execute a trade, use the following function on the ``portfolio`` argument.

```py
portfolio.order('AAPL', 100) # buy 100 APPL
portfolio.order('GOOG', -50) # sell 50 GOOG
```

``portfolio.order`` will return ``True`` if successful and ``False`` if not. If a trade will exceed your delta limit, it will fail.

Some useful variables:

```py
portfolio.delta: dict           # current delta
portfolio.delta_limits: dict    # delta limits (set by backtester)
portfolio.stocks: list          # available stocks
```

Note, the backtester will return your delta to 0 (including fees if applicable).

## The Backtester

In [8]:
from qfinuwa import Backtester

In [53]:
backtester = Backtester(MyStrategy, CustomIndicators, ['VGT'], data_folder='data', days=30, fee=0.01, delta_limits=100)

# backtester = Backtester(MyStrategy, CustomIndicators, ['AAPL','MSFT','GOOGL','NVDA'], 
#                         data_folder='data', days='all', fee=0.01)

> Precompiling data: 100%|██████████| 431430/431430 [00:03<00:00, 116007.11it/s]


The backtester may take a few seconds to create itself, but once it is created you shouldn't do it again unless you want to change the data. It takes a long time to create itself because it is precompiling the data for iterating.

``delta_limits`` can be dictionary of specific delta limits for each stock, or an integer that will apply to all stocks.

### Changing Backtesting Parameters

To see the current parameters of the backtester, run the following cell.

In [50]:
backtester

Backtester:
- Strategy: MyStrategy
	- Params: {'quantity': 5}
- Indicators: CustomIndicators
	- Params: {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}}
	- SingleIndicators: []
	- MultiIndicators: ['lower_bollinger', 'upper_bollinger']
- Stocks: ['VGT']
- Fee 0.01
- delta_limit: {'VGT': 100}
- Days: 30

To change the parameters.

In [12]:
# backtester.fee = 0.002
backtester.delta_limits = {'AAPL': 100, 'GOOGL': 100, 'MSFT': 200, 'NVDA': 200} # different delta limits
# backtester.delta_limits = 100
backtester.days = 30

backtester

Backtester:
- Strategy: MyStrategy
	- Params: {'quantity': 5}
- Indicators: CustomIndicators
	- Params: {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}
	- SingleIndicators: ['etf']
	- MultiIndicators: ['lower_bollinger', 'upper_bollinger']
- Stocks: ['AAPL', 'GOOGL', 'MSFT', 'NVDA']
- Fee 0.01
- delta_limit: {'AAPL': 100, 'GOOGL': 100, 'MSFT': 200, 'NVDA': 200}
- Days: 30

## Changing Strategy Parameters

In [13]:
print('current params', backtester.strategy.params)
print('default params', backtester.strategy.defaults)

current params {'quantity': 5}
default params {'quantity': 5}


In [14]:
backtester.strategy.update_params({'quantity': 10})

In [15]:
backtester.strategy = MyStrategy # update strategy (for example if you change your code)
# backtester.strategy = RandomStrategy

## Changing Indicator Parameters

In [16]:
print('current params', backtester.indicators.params)
print('default params', backtester.indicators.defaults)

current params {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}
default params {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}


In [17]:
backtester.indicators.update_params({'etf': {'a': 0.0}, 'bollinger_bands': {'BOLLINGER_WIDTH': 1, 'WINDOW_SIZE': 50}})
backtester.indicators.params

{'bollinger_bands': {'BOLLINGER_WIDTH': 1, 'WINDOW_SIZE': 50},
 'etf': {'a': 0.0, 'b': 0.2, 'c': 0.05, 'd': 0.05}}

In [18]:
backtester.indicators = CustomIndicators
backtester.indicators.params

{'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100},
 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}

## Running Backtests

To run a backtest, use ``Backtester.run``.

``run`` will pick ``cv`` random periods of ``backtester.days`` days and run your strategy on that window of data. If you want reproducibility, use ``seed`` to use the same random generation every time, else ommit it as a key word argument.

In [54]:
output = backtester.run(cv=3)

> Running backtest over 3 samples of 30 days: 100%|██████████| 3/3 [00:01<00:00,  2.75it/s]


In [56]:
print(output)


{'strategy': {'quantity': 5}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}}}

Mean ROI:	-23967.39967618871
STD ROI:	3737.3363669523687

09/11/2022 -> 08/12/2022:	-19938.239
17/09/2021 -> 15/10/2021:	-23019.562
13/08/2021 -> 10/09/2021:	-28944.398

AVERAGED RESULTS FOR 3 RUNS:
|               |      VGT |      Net |
|---------------|----------|----------|
| n_trades      |  1441.67 |  1441.67 |
| n_buys        |  721.667 |  721.667 |
| n_sells       |      720 |      720 |
| gross_pnl     |  3887.23 |  3887.23 |
| fees_paid     |  27854.6 |  27854.6 |
| net_pnl       | -23967.4 | -23967.4 |
| pnl_per_trade | -16.5656 | -16.5656 |


In [24]:
print("Mean ROI and STD ROI: ", output.roi)

Mean ROI and STD ROI:  (-107356.72752241712, 12662.755778301274)


In [27]:
print('4th run PNL is ', output[3].roi)

4th sample PNL is  -95056.29197593084


In [32]:
pnl = []
for run in output:
    pnl.append(run.roi)
pnl

# [run.roi for run in output]

[-128683.49139413793,
 -101344.32437314373,
 -97049.06940741782,
 -95056.29197593084,
 -114650.46046145516]

In [34]:
# Example of obtaining custom runtime data (see Strategy.on_finish)
print('# failed orders:', [run.on_finish for run in output])

# Get parameters 
print(output.parameters)

# failed orders: [5379, 10319, 7348, 5808, 7765]
{'strategy': {'quantity': 10}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}}


Or you can specify which dates to start (the end date will be specified by ``backtester.days``).

Note, not all days will contain data - the market may be closed.

In [35]:

print(backtester.date_range, backtester.days)
output = backtester.run(cv=5, start_dates=['2021-06-07', '2021-04-19', '2022-01-01'])

print(output)

(Timestamp('2021-04-09 09:31:00'), Timestamp('2022-12-29 16:00:00')) 30


> Running backtest over 3 samples of 30 days: 100%|██████████| 3/3 [00:03<00:00,  1.08s/it]


{'strategy': {'quantity': 10}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.2, 'b': 0.2, 'c': 0.05, 'd': 0.05}}}

Mean ROI:	-126122.6843619074
STD ROI:	10767.184946186198

06/07/2021 -> 04/08/2021:	-127970.443
19/04/2021 -> 18/05/2021:	-112109.200
03/01/2022 -> 28/01/2022:	-138288.409

AVERAGED RESULTS FOR 3 RUNS:
|               |     AAPL |    GOOGL |     MSFT |     NVDA |      Net |
|---------------|----------|----------|----------|----------|----------|
| n_trades      |  1438.67 |     1297 |     2098 |     2015 |  6848.67 |
| n_buys        |      718 |      647 |  1043.33 |  1004.33 |  3412.67 |
| n_sells       |  720.667 |      650 |  1054.67 |  1010.67 |     3436 |
| gross_pnl     |  474.477 | -4105.02 | -816.447 |  2119.41 | -2327.58 |
| fees_paid     |  21166.9 |    35832 |  26637.2 |    40159 |   123795 |
| net_pnl       | -20692.4 |   -39937 | -27453.7 | -38039.6 |  -126123 |
| pnl_per_trade | -14.3916 | -31.2764 | -13.1583 | -

You can either use ``Backtester.strategy.update_params`` to update parameters, or just pass them into ``run``.

In [ ]:
S = {
    'quantity': 1
}

I = {
    'bollinger_bands': {
        'BOLLINGER_WIDTH': 1,
        'WINDOW_SIZE': 50
    },
    'etf': {
        'a': 0.0
    }
}


output = backtester.run(strategy_params = S, indicator_params = I, cv=5)

## Hyper-Parameter Sweep

To run a basic grid-search over all your hyperparameters, use ``Backtester.run_grid_search``.

In [36]:
output = backtester.run_grid_search(
    strategy_params={'quantity': [1, 10]},
    indicator_params={'bollinger_bands': {
                            'BOLLINGER_WIDTH': [1, 2], 
                            }, 
                        'etf': {'a': [0.0, 0.5]}},
    cv=2)

> Backtesting the across the following ranges:
Agorithm Parameters {'quantity': [1, 10]}
Indicator Parameters {'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': 100}, 'etf': {'a': [0.0, 0.5], 'b': 0.2, 'c': 0.05, 'd': 0.05}}


Running paramter sweep (cv=2): 100%|██████████| 8/8 [00:14<00:00,  1.84s/it]


In [37]:
print(output.parameters)
output

({'quantity': [1, 10]}, {'bollinger_bands': {'BOLLINGER_WIDTH': [1, 2], 'WINDOW_SIZE': 100}, 'etf': {'a': [0.0, 0.5], 'b': 0.2, 'c': 0.05, 'd': 0.05}})


Best parameter results:

{'strategy': {'quantity': 1}, 'indicator': {'bollinger_bands': {'BOLLINGER_WIDTH': 2, 'WINDOW_SIZE': 100}, 'etf': {'a': 0.0, 'b': 0.2, 'c': 0.05, 'd': 0.05}}}

Mean ROI:	-24544.428164563855
STD ROI:	3187.2294041379773

04/08/2021 -> 02/09/2021:	-21357.199
22/09/2021 -> 21/10/2021:	-27731.658

AVERAGED RESULTS FOR 2 RUNS:
|               |     AAPL |    GOOGL |     MSFT |     NVDA |      Net |
|---------------|----------|----------|----------|----------|----------|
| n_trades      |   2992.5 |   3015.5 |   3294.5 |     3170 |  12472.5 |
| n_buys        |   1467.5 |   1525.5 |   1632.5 |   1525.5 |     6151 |
| n_sells       |     1525 |     1490 |     1662 |   1644.5 |   6321.5 |
| gross_pnl     |   73.161 | -922.641 | -889.219 |  1524.19 | -214.509 |
| fees_paid     |  4328.39 |  8759.35 |  4569.79 |  6672.39 |  24329.9 |
| net_pnl       | -4255.23 | -9681.99 | -5459.01 |  -5148.2 | -24544.4 |
| pnl_per_trade | -1.42072 | -3.20171 | -1.65708 | -1.60928 | -1.966

## Plotting

In [38]:
from qfinuwa import Plotting

In [39]:
output = backtester.run(cv=3)
Plotting.plot_result(output[0], stocks=['NVDA', 'MSFT'], show_transactions = True, show_portfolio=False)

> Running backtest over 3 samples of 30 days: 100%|██████████| 3/3 [00:04<00:00,  1.60s/it]
